<a href="https://colab.research.google.com/github/shekharkhushi98/Flight-Price-Prediction/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===== 1: Install required packages =====
!pip install linearmodels --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 7.5 MB/s eta 0:00:00


In [2]:
# ===== 2: Imports =====
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from scipy import stats
from linearmodels.iv import IV2SLS
from google.colab import files


In [3]:
# ===== 3: Upload dataset =====
print("Please upload Clean_Dataset.csv using the file picker.")
uploaded = files.upload()  # choose Clean_Dataset.csv from your computer
fn = list(uploaded.keys())[0]
print("Uploaded:", fn)

Please upload Clean_Dataset.csv using the file picker.


Saving Clean_Dataset.csv to Clean_Dataset.csv
Uploaded: Clean_Dataset.csv


In [4]:

# ===== 4: Load and quick inspect =====
df = pd.read_csv(fn)
print("Rows, cols:", df.shape)
display(df.head())

Rows, cols: (300153, 12)


,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [5]:
# ===== 5: Ensure required columns exist & basic cleaning =====
required_cols = ['price','duration','stops','airline','source_city','destination_city',
                 'departure_time','arrival_time','days_left','class']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Dataset missing columns: {missing}")

# Keep only required cols and drop NA
df = df[required_cols].dropna().copy()
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
df['days_left'] = pd.to_numeric(df['days_left'], errors='coerce')
df = df.dropna(subset=['duration','days_left'])  # ensure numeric columns are ok

# Standardize stops labels to known categories
# We expect categories like 'zero', 'one', 'two_or_more' or similar.
df['stops'] = df['stops'].astype(str)

In [6]:
# ===== 6: Create exogenous stop dummies per your spec =====
# Create dummies for stops but keep only 'stops_zero' and 'stops_two_or_more' as exogenous controls.
# We'll treat the omitted category as 'one_stop' (or whatever the remaining category is).
stops_dummies = pd.get_dummies(df['stops'], prefix='stops')
# Identify or create standardized names:
# Keep 'stops_zero' and 'stops_two_or_more' if present; otherwise infer closest labels.
# We'll pick first and last as zero and two_or_more if names differ.
# Preferred explicit names:
if 'stops_zero' not in stops_dummies.columns:
    # try to map common labels
    for col in stops_dummies.columns:
        if 'zero' in col.lower():
            stops_dummies.rename(columns={col: 'stops_zero'}, inplace=True)
        elif 'two' in col.lower() or 'more' in col.lower():
            stops_dummies.rename(columns={col: 'stops_two_or_more'}, inplace=True)
# If still missing, create columns with 0 (safe fallback)
if 'stops_zero' not in stops_dummies.columns:
    stops_dummies['stops_zero'] = 0
if 'stops_two_or_more' not in stops_dummies.columns:
    # create from existing categories if possible
    possible = [c for c in stops_dummies.columns if ('two' in c.lower() or 'more' in c.lower())]
    if possible:
        stops_dummies['stops_two_or_more'] = stops_dummies[possible].sum(axis=1).clip(upper=1)
    else:
        stops_dummies['stops_two_or_more'] = 0

# Exogenous control matrix (const + stops_zero + stops_two_or_more)
exog_controls = pd.DataFrame({
    'const': 1.0,
    'stops_zero': stops_dummies['stops_zero'].astype(float),
    'stops_two_or_more': stops_dummies['stops_two_or_more'].astype(float)
})

In [10]:
# ===== 7: Build instrument matrix Z =====
# Instruments: airline, source_city, destination_city, departure_time, arrival_time, class, days_left
instr_cats = ['airline','source_city','destination_city','departure_time','arrival_time','class']
# OneHotEncode each categorical instrument with drop='first' to avoid collinearity
Z_parts = [exog_controls]  # include exogenous controls (valid instruments too)
Z_names = list(exog_controls.columns)

for cat in instr_cats:
    enc = OneHotEncoder(drop='first')
    arr = enc.fit_transform(df[[cat]]).toarray() # Explicitly convert to dense array
    names = [f"{cat}_{val}" for val in enc.categories_[0][1:]]
    Z_parts.append(pd.DataFrame(arr, columns=names))
    Z_names.extend(names)

# Add days_left (numeric instrument)
Z_parts.append(df[['days_left']].reset_index(drop=True))
Z_names.append('days_left')

# Final instrument matrix Z (n x L)
Z = pd.concat(Z_parts, axis=1).reset_index(drop=True)
Z.columns = Z_names
# Sanity
n = Z.shape[0]
L = Z.shape[1]
print("Instruments L =", L)

# ===== Cell 8: Build regressor matrix X and outcome y =====
# X = [const, duration (endogenous), stops_zero, stops_two_or_more]
y = df['price'].values.reshape(-1,1)
duration = df['duration'].values.reshape(-1,1)  # endogenous

# Compose X as numpy array in order: intercept, duration, stops_zero, stops_two_or_more
X = np.hstack([
    exog_controls['const'].values.reshape(-1,1),
    duration,
    exog_controls['stops_zero'].values.reshape(-1,1),
    exog_controls['stops_two_or_more'].values.reshape(-1,1)
])
X_cols = ['const','duration','stops_zero','stops_two_or_more']
k = X.shape[1]
print("Observations n =", n, ", regressors k =", k)


Instruments L = 30
Observations n = 300153 , regressors k = 4


In [11]:
# ===== 9: GMM helper functions (linear GMM) =====
def gmm_beta(X, Z, y, W):
    A = X.T @ Z @ W @ Z.T @ X
    b = X.T @ Z @ W @ Z.T @ y
    beta = np.linalg.solve(A, b)
    return beta

def gbar_and_resid(X, Z, y, beta):
    resid = y - X @ beta
    gbar = (Z.T @ resid) / n
    return gbar, resid

def compute_S(Z, resid):
    Uz = Z.values * resid.flatten()[:,None]  # elementwise
    S = (Uz.T @ Uz) / n
    return S

def sandwich_var(beta, X, Z, W, S):
    A = (X.T @ Z.values @ W @ Z.values.T @ X) / n
    B = (X.T @ Z.values @ W @ S @ W @ Z.values.T @ X) / n
    Ainv = np.linalg.inv(A)
    Var = (Ainv @ B @ Ainv) / n
    return Var


In [12]:
# ===== 10: 1-step GMM (W = (Z'Z)^{-1}) =====
W1 = np.linalg.inv(Z.values.T @ Z.values)
beta1 = gmm_beta(X, Z.values, y, W1)  # (k x 1)
gbar1, resid1 = gbar_and_resid(X, Z.values, y, beta1)
S1 = compute_S(Z, resid1)
Var_beta1 = sandwich_var(beta1, X, Z, W1, S1)
se_beta1 = np.sqrt(np.diag(Var_beta1)).reshape(-1,1)

print("\n=== 1-step GMM results ===")
for name, b, se in zip(X_cols, beta1.flatten(), se_beta1.flatten()):
    t = b / se
    p = 2 * (1 - stats.t.cdf(np.abs(t), df=n - k))
    print(f"{name:18s} coef={b:12.4f}  se={se:8.4f}  t={t:10.3f}  p={p:8.4f}")

# 1-step Hansen J
J1 = n * (gbar1.T @ W1 @ gbar1).item()
df_J = L - k
pval_J1 = 1 - stats.chi2.cdf(J1, df_J)
print(f"\n1-step Hansen J = {J1:.4f}, df = {df_J}, p-value = {pval_J1:.6f}")



=== 1-step GMM results ===
const              coef=    322.2100  se=  0.2423  t=  1329.805  p=  0.0000
duration           coef=   1672.9343  se=  0.0190  t= 87859.467  p=  0.0000
stops_zero         coef=   5387.7873  se=  0.2343  t= 22997.125  p=  0.0000
stops_two_or_more  coef= -11833.3297  se=  0.3541  t=-33422.256  p=  0.0000

1-step Hansen J = 419610179.9353, df = 26, p-value = 0.000000


In [13]:

# ===== 11: 2-step (efficient) GMM =====
# Step 2 weight matrix = S1^{-1} (from residuals)
W2 = np.linalg.inv(S1 + 1e-12*np.eye(S1.shape[0]))  # small regularizer for numeric stability
beta2 = gmm_beta(X, Z.values, y, W2)
gbar2, resid2 = gbar_and_resid(X, Z.values, y, beta2)
S2 = compute_S(Z, resid2)
W2_final = np.linalg.inv(S2 + 1e-12*np.eye(S2.shape[0]))
beta2_final = gmm_beta(X, Z.values, y, W2_final)
gbar2_final, resid2_final = gbar_and_resid(X, Z.values, y, beta2_final)
S_final = compute_S(Z, resid2_final)
Var_beta2 = sandwich_var(beta2_final, X, Z, W2_final, S_final)
se_beta2 = np.sqrt(np.diag(Var_beta2)).reshape(-1,1)

print("\n=== 2-step (efficient) GMM results ===")
for name, b, se in zip(X_cols, beta2_final.flatten(), se_beta2.flatten()):
    t = b / se
    p = 2 * (1 - stats.t.cdf(np.abs(t), df=n - k))
    print(f"{name:18s} coef={b:12.4f}  se={se:8.4f}  t={t:10.3f}  p={p:8.4f}")

# 2-step Hansen J
J2 = n * (gbar2_final.T @ W2_final @ gbar2_final).item()
pval_J2 = 1 - stats.chi2.cdf(J2, df_J)
print(f"\n2-step Hansen J = {J2:.4f}, df = {df_J}, p-value = {pval_J2:.6f}")


=== 2-step (efficient) GMM results ===
const              coef=  -3246.7578  se=  0.1145  t=-28358.282  p=  0.0000
duration           coef=   1090.3351  se=  0.0101  t=107817.365  p=  0.0000
stops_zero         coef=   7633.5168  se=  0.1659  t= 46014.808  p=  0.0000
stops_two_or_more  coef= -15870.1995  se=  0.3889  t=-40812.281  p=  0.0000

2-step Hansen J = 115392.8536, df = 26, p-value = 0.000000


In [14]:
# ===== 12: First-stage regressions & weak instrument check =====
# First-stage: regress endogenous (duration) on ALL instruments (Z) and exogenous controls.
# We'll run OLS of duration on Z (which already contains exog controls)
fs_model = sm.OLS(df['duration'].values, Z).fit(cov_type='HC1')
print("\n=== First-stage summary (duration ~ instruments + exog_controls) ===")
print(fs_model.summary())

# Compute joint F-statistic for excluded instruments:
# Excluded instruments = Z columns excluding the exogenous controls (const, stops_zero, stops_two_or_more)
excluded = [c for c in Z.columns if c not in ['const','stops_zero','stops_two_or_more']]
# Restricted regression (only exog controls)
Z_restricted = Z[['const','stops_zero','stops_two_or_more']]
res_restricted = sm.OLS(df['duration'].values, Z_restricted).fit()
RSS_r = sum(res_restricted.resid**2)
RSS_ur = sum(fs_model.resid**2)
q = len(excluded)
df_r = Z_restricted.shape[1]
df_ur = fs_model.df_model + 1  # params count
F_stat = ((RSS_r - RSS_ur)/q) / (RSS_ur / (n - int(df_ur)))
pval_F = 1 - stats.f.cdf(F_stat, q, n - int(df_ur))
print(f"\nFirst-stage joint F-statistic for excluded instruments = {F_stat:.3f}, q = {q}, p = {pval_F:.6f}")



=== First-stage summary (duration ~ instruments + exog_controls) ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.465
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                 1.472e+04
Date:                Sun, 30 Nov 2025   Prob (F-statistic):               0.00
Time:                        18:14:07   Log-Likelihood:            -9.2413e+05
No. Observations:              300153   AIC:                         1.848e+06
Df Residuals:                  300123   BIC:                         1.849e+06
Df Model:                          29                                         
Covariance Type:                  HC1                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------

In [15]:
# ===== 13: IV 2SLS (for comparison) =====
# Build DataFrame for IV2SLS (linearmodels expects arrays)
iv_df = pd.DataFrame({
    'price': df['price'].values,
    'duration': df['duration'].values,
    'const': 1.0,
    'stops_zero': exog_controls['stops_zero'].values,
    'stops_two_or_more': exog_controls['stops_two_or_more'].values
})
# add excluded instruments to iv_df
for col in excluded:
    iv_df[col] = Z[col].values

exog_list = ['const','stops_zero','stops_two_or_more']
instr_list = excluded

iv = IV2SLS(dependent=iv_df['price'],
            exog=iv_df[exog_list],
            endog=iv_df[['duration']],
            instruments=iv_df[instr_list]).fit(cov_type='robust')
print("\n=== IV 2SLS results (for comparison) ===")
print(iv.summary)



=== IV 2SLS results (for comparison) ===
                          IV-2SLS Estimation Summary                          
Dep. Variable:                  price   R-squared:                     -0.0457
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0457
No. Observations:              300153   F-statistic:                 5.113e+04
Date:                Sun, Nov 30 2025   P-value (F-stat)                0.0000
Time:                        18:14:20   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                 322.21     132.75     2.4273     0.0152      6

In [16]:
# ===== 14: Summary table for assignment =====
print("\n=== Summary table (1-step vs 2-step GMM vs 2SLS) ===")
import pandas as pd
table = pd.DataFrame({
    'coef_1step': beta1.flatten(),
    'se_1step': se_beta1.flatten(),
    'coef_2step': beta2_final.flatten(),
    'se_2step': se_beta2.flatten()
}, index=X_cols)
display(table)


=== Summary table (1-step vs 2-step GMM vs 2SLS) ===


,coef_1step,se_1step,coef_2step,se_2step
const,322.210045,0.242299,-3246.757780,0.114491
duration,1672.934338,0.019041,1090.335106,0.010113
stops_zero,5387.787291,0.234281,7633.516803,0.165893
stops_two_or_more,-11833.329738,0.354055,-15870.199505,0.388858


In [17]:
# ===== 15: Short interpretation template (paste into your assignment) =====
print("\n=== Interpretation template ===\n")
print("Model: price = beta0 + beta1*duration (endogenous) + beta2*stops_zero + beta3*stops_two_or_more + u")
print("Instruments: airline dummies, source/destination/datetime dummies, class, days_left (plus exogenous stops dummies).")
print("Report coefficients (see table). 2-step GMM is asymptotically efficient under heteroskedasticity.")
print("Hansen J p-value tests overidentifying restrictions (L-k df). First-stage F tests for weak IV (F > 10 is good).")



=== Interpretation template ===

Model: price = beta0 + beta1*duration (endogenous) + beta2*stops_zero + beta3*stops_two_or_more + u
Instruments: airline dummies, source/destination/datetime dummies, class, days_left (plus exogenous stops dummies).
Report coefficients (see table). 2-step GMM is asymptotically efficient under heteroskedasticity.
Hansen J p-value tests overidentifying restrictions (L-k df). First-stage F tests for weak IV (F > 10 is good).
